In [ ]:
# 코랩에서만 실행
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
##페이지 입력해서 키워드로 크롤링
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import os #-----------------------------------------------------------------------수정

#크롤링 csv파일 저장위치
#/content/drive/MyDrive/[텍스트분석] 팀 프로젝트/dataset
#공유 드라이브-> 드라이브에 바로가기 추가
f_path = '/content/drive/MyDrive/[텍스트분석] 팀 프로젝트/dataset' #------------수정

In [4]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

In [5]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [8]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#NULL값 제거 및 전처리 --------------------------------------------------------------수정
news_df = news_df.dropna()
news_df = news_df.drop_duplicates()
news_df['content'] = news_df['content'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
news_df['content'] = news_df['content'].str.replace(r'[0-9]+', repl=r' ', regex=True)

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('/content/drive/MyDrive/{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),
                            encoding='utf-8-sig',index=False) #-----------------------------------수정 (csv 파일 저장 위치 추가)

검색할 키워드를 입력해주세요:코로나

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):20

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=91', 'https://search.naver.com/search.nav

100%|██████████| 169/169 [00:41<00:00,  4.03it/s]

검색된 기사 갯수: 총  200 개

[뉴스 제목]
['국정원 "탈북 가족, 남한 방송 보며 동경…코로나 통제에 귀순"', '코로나 신규 확진 1만9586명…중증 144명·사망 13명', "[Q&amp;A]코로나 위기단계 내려가면 '비대면 진료' 어떻게 되나", '미 단체 "코로나19 이후 새터민 급감…지난해 67명뿐"', '국제선 운항 횟수 642%↑...9월 코로나 이전 수준 회복', '[그래픽] 코로나19 확진자 지역별 현황(19일)', '[세계를 가다]코로나 빗장 풀고 보니…‘중국’ 된 홍콩', '코로나가 학교급식도 바꿨다…"삼계탕 줄고, 라면 늘고"', '코로나 한창일 때 골프 친 대책단장…자가격리되자 "마트 가서"', "코로나 사라지자 다시 찾아온 봄철 '영유아 수족구병'", '국제선 운항 횟수 642%↑...9월 코로나 이전 수준 회복', '코로나19 신규확진 2만1797명… 1주 전보다 1200여명 많아', '코로나 끝나도 사무실 ‘텅’…디폴트 급증', '여성 주식 부호 톱30, 코로나19 기간 평가액 90% 상승', '"중국 코로나 발병 시점에 우한연구소 바이러스 유출"', '[그래픽] 코로나19 확진자 지역별 현황(19일)', '충북 어제 624명 코로나 확진…1주일 전보다 80명↑', '경북, 732명 코로나19 확진…모든 시군에서 발생', '인천 1147명 코로나19 확진…감염자 1명 사망', '[세계를 가다]코로나 빗장 풀고 보니…‘중국’ 된 홍콩', '코로나19 신규확진 1만9586명…1주 전보다 400여명 감소(종합)', '코로나 이전 대비 국제선 운항 65%, 공항인력 90% 회복', '코로나 신규확진 2만6147명…중증 147명·사망 11명', '경남, 어제 923명 코로나19 확진…사망 2명 추가', '아이까지 데리고 위험한 루트로 귀순…“北식량난·코로나 봉쇄 피해 탈출한 듯”', "서울대 '코로나19 교훈과 미래 대비' 포럼 22일 개최", '국제선 운항 1년간 642% 증가…코로나 이전 65% 수준 회복', '경남교육청 "코

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
news=pd.read_csv('/content/drive/MyDrive/[텍스트분석] 팀 프로젝트/dataset/파산_20230417_08시51분04초.csv')
news

,date,title,link,content
0,2023-04-16 06:40:01,"""파산·전액인출""…금감원, '위기설' 악용 불공정거래 집중감시",https://n.news.naver.com/mnews/article/001/001...,공매도 세력 등과 연계 가능성 조사 허위사실 유포에는 법적 조치 불공정거...
1,2023-04-17 14:03:19,SVB 파산 후 벼랑 끝 몰린 美 중소형 지역은행...수신금리 올려도 고객 떠난다,https://n.news.naver.com/mnews/article/014/000...,월말 현재 미국 대 은행 예금 억 증가 나머지 중소형 은행은 억 달러 감...
2,2023-04-17 17:17:40,"올해 1분기 벤처 투자 60% 급감...""실리콘밸리 파산 여파""",https://n.news.naver.com/mnews/article/052/000...,미국 실리콘밸리은행 SVB 파산 여파 등 글로벌 금융 시장 불안이 커지면...
3,2023-04-17 05:04:51,금감원 ‘저축은행 파산’ 찌라시 엄단,https://n.news.naver.com/mnews/article/081/000...,허위사실 유포 고발해 형사처벌공매도 세력 등 연계 가능성 조사최근 대형 저축은행...
4,2023-04-17 05:03:06,"SVB 파산 사태 전부터, 미국 경제는 하강하고 있었다",https://n.news.naver.com/mnews/article/028/000...,Weconomy 공동락의 경제 스토리 일 현지시각 예금 인출 위해 ...
5,2023-04-15 14:18:30,"이창용, SVB 파산 사태에 ""디지털 속도 대비… 지급보증 담보자산 상향 검토""",https://n.news.naver.com/mnews/article/417/000...,이창용 한국은행 총재가 일 서울 중구 한국은행에서 열린 월 금융통화위...
6,2023-04-12 20:36:35,"머스크 ""트위터 운영, 매우 고통스러워…지금도 거의 파산 상태""",https://n.news.naver.com/mnews/article/003/001...,기사내용 요약가끔 사무실 소파서 자며 대량해고 단행 등 변화 주도비용 못 줄이면...
7,2023-04-13 11:02:12,SVB 파산 후 미국 은행 시스템 불안정 하지만 금융위기로 번지지 않을 것,https://n.news.naver.com/mnews/article/014/000...,워런 버핏 파산하는 미국 은행 더 나올 수 있어 백악관 NEC 위원장 미국 ...
8,2023-04-13 00:15:01,"워런 버핏 ""은행 실패, 아직 안끝났다"" 추가 파산 경고",https://n.news.naver.com/mnews/article/277/000...,투자의 귀재 로 불리는 워런 버핏 버크셔해서웨이 회장이 최근 실리콘밸리은...
9,2023-04-12 21:25:01,실리콘밸리 은행 파산이 불러일으킨 나비효과,https://n.news.naver.com/mnews/article/047/000...,출장 무산으로 창덕궁 봄꽃 컬렉션이 되어 버린 도쿄 컬렉션 오마이뉴스의 모토는 ...


In [ ]:
#뉴스 카테고리
news['category']=0
for i in range(0,14):
 
  news['category'][i]=(news.link[i][-3:])
  i=i+1


<ipython-input-15-18455f6573b2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['category'][i]=(news.link[i][-3:])


In [ ]:
news


,date,title,link,content,category
0,2023-04-16 06:40:01,"""파산·전액인출""…금감원, '위기설' 악용 불공정거래 집중감시",https://n.news.naver.com/mnews/article/001/001...,공매도 세력 등과 연계 가능성 조사 허위사실 유포에는 법적 조치 불공정거...,101
1,2023-04-17 14:03:19,SVB 파산 후 벼랑 끝 몰린 美 중소형 지역은행...수신금리 올려도 고객 떠난다,https://n.news.naver.com/mnews/article/014/000...,월말 현재 미국 대 은행 예금 억 증가 나머지 중소형 은행은 억 달러 감...,101
2,2023-04-17 17:17:40,"올해 1분기 벤처 투자 60% 급감...""실리콘밸리 파산 여파""",https://n.news.naver.com/mnews/article/052/000...,미국 실리콘밸리은행 SVB 파산 여파 등 글로벌 금융 시장 불안이 커지면...,101
3,2023-04-17 05:04:51,금감원 ‘저축은행 파산’ 찌라시 엄단,https://n.news.naver.com/mnews/article/081/000...,허위사실 유포 고발해 형사처벌공매도 세력 등 연계 가능성 조사최근 대형 저축은행...,101
4,2023-04-17 05:03:06,"SVB 파산 사태 전부터, 미국 경제는 하강하고 있었다",https://n.news.naver.com/mnews/article/028/000...,Weconomy 공동락의 경제 스토리 일 현지시각 예금 인출 위해 ...,101
5,2023-04-15 14:18:30,"이창용, SVB 파산 사태에 ""디지털 속도 대비… 지급보증 담보자산 상향 검토""",https://n.news.naver.com/mnews/article/417/000...,이창용 한국은행 총재가 일 서울 중구 한국은행에서 열린 월 금융통화위...,101
6,2023-04-12 20:36:35,"머스크 ""트위터 운영, 매우 고통스러워…지금도 거의 파산 상태""",https://n.news.naver.com/mnews/article/003/001...,기사내용 요약가끔 사무실 소파서 자며 대량해고 단행 등 변화 주도비용 못 줄이면...,104
7,2023-04-13 11:02:12,SVB 파산 후 미국 은행 시스템 불안정 하지만 금융위기로 번지지 않을 것,https://n.news.naver.com/mnews/article/014/000...,워런 버핏 파산하는 미국 은행 더 나올 수 있어 백악관 NEC 위원장 미국 ...,101
8,2023-04-13 00:15:01,"워런 버핏 ""은행 실패, 아직 안끝났다"" 추가 파산 경고",https://n.news.naver.com/mnews/article/277/000...,투자의 귀재 로 불리는 워런 버핏 버크셔해서웨이 회장이 최근 실리콘밸리은...,101
9,2023-04-12 21:25:01,실리콘밸리 은행 파산이 불러일으킨 나비효과,https://n.news.naver.com/mnews/article/047/000...,출장 무산으로 창덕궁 봄꽃 컬렉션이 되어 버린 도쿄 컬렉션 오마이뉴스의 모토는 ...,103


In [ ]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from kafka import KafkaProducer
import time
from json import dumps

news_url_numbers = ['437','020','016','057','028','005','023','032','214','366','277','448','079','088','449']
# news_url_numbers = ['421','056','025','003','055','047','052','001','015','215','422','469','081','018','014','022']

# 각 16개 총 32개

producer = KafkaProducer(
    acks=0,
    compression_type='gzip',
    bootstrap_servers=['125.133.75.24:33149'],
    value_serializer=lambda v: dumps(v).encode('utf-8'),
    api_version=(0,11,5)
)

In [ ]:
news_type_numbers = ['154', '100', '101', '102', '103', '104']
news_type = ['대선', '정치', '경제', '사회', '생활', '세계']
remove_text = '''.,'"“”‘’[]?!'''
space_text = '·…'
title_list = ['' for num in range(0,15)]

In [ ]:
while True:

    start = time.time()

    for news_url_number in news_url_numbers:
        news_type_num = 0
        for news_type_number in news_type_numbers:
            response = requests.get('https://media.naver.com/press/' + news_url_number + '?sid=' + news_type_number)

            if response.status_code == 200:
                # time.sleep(0.5)
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')

                news_titles = soup.select('.press_edit_news_title')
                news_times = soup.select('.r_ico_b.r_modify ')
                news_urls = soup.select('.press_edit_news_link._es_pc_link')
                news_company = soup.select('.press_hd_name_link')[0].text.strip()

                for i in range(0, 2):
                    try:
                        news_title = news_titles[i].text
                        news_url = news_urls[i].attrs['href']
                        news_time = news_times[i].text
                    except:
                        continue

                    if news_url.split('/')[5][:-8] in title_list:
                        continue

                    if '시간전' in news_time or '일전' in  news_time or '2022' in  news_time:
                        continue

                    if int(news_time.split('분전')[0]) < 2:  # 몇분전 데이터를 >가져올 것인지
                        del title_list[0]
                        title_list.append(news_url.split('/')[5][:-8])
                        # print(title_list)

                        for remove in remove_text:
                            news_title = news_title.replace(remove, '')
                        for space in space_text:
                            news_title = news_title.replace(space, ' ')

                        # print(news_title, news_url, news_time, news_company, news_type[news_type_num])
                        # print('***\n')

                        producer.send('news', {
                            'title': news_title,
                            'news_company': news_company,
                            'news_url': news_url,
                            'news_type': news_type[news_type_num],
                        })
                        producer.flush()
            news_type_num += 1

    print("time :", time.time() - start)

ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=125.133.75.24:33149 <connecting> [IPv4 ('125.133.75.24', 33149)]> timed out


KeyboardInterrupt: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup
from kafka import KafkaProducer
import time
from json import dumps


news_url_numbers = ['437','020','016','057','028','005','023','032','214','366','277','448','079','088','449']
# news_url_numbers = ['421','056','025','003','055','047','052','001','015','215','422','469','081','018','014','022']

# 각 16개 총 32개

producer = KafkaProducer(
    acks=0,
    compression_type='gzip',
    bootstrap_servers=['125.133.75.24:33149'],
    value_serializer=lambda v: dumps(v).encode('utf-8'),
    api_version=(0,11,5) #----------------------------------------------------------------수정
)

news_type_numbers = ['154', '100', '101', '102', '103', '104']
news_type = ['대선', '정치', '경제', '사회', '생활', '세계']
remove_text = '''.,'"“”‘’[]?!'''
space_text = '·…'
title_list = ['' for num in range(0,15)]


while True:

    start = time.time()

    for news_url_number in news_url_numbers:
        news_type_num = 0
        for news_type_number in news_type_numbers:
            response = requests.get('https://media.naver.com/press/' + news_url_number + '?sid=' + news_type_number)

            if response.status_code == 200:
                # time.sleep(0.5)
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')

                news_titles = soup.select('.press_edit_news_title')
                news_times = soup.select('.r_ico_b.r_modify ')
                news_urls = soup.select('.press_edit_news_link._es_pc_link')
                news_company = soup.select('.press_hd_name_link')[0].text.strip()

                for i in range(0, 2):
                    try:
                        news_title = news_titles[i].text
                        news_url = news_urls[i].attrs['href']
                        news_time = news_times[i].text
                    except:
                        continue

                    if news_url.split('/')[5][:-8] in title_list:
                        continue

                    if '시간전' in news_time or '일전' in  news_time or '2022' in  news_time:
                        continue

                    if int(news_time.split('분전')[0]) < 2:  # 몇분전 데이터를 >가져올 것인지
                        del title_list[0]
                        title_list.append(news_url.split('/')[5][:-8])
                        # print(title_list)

                        for remove in remove_text:
                            news_title = news_title.replace(remove, '')
                        for space in space_text:
                            news_title = news_title.replace(space, ' ')

                        # print(news_title, news_url, news_time, news_company, news_type[news_type_num])
                        # print('***\n')

                        producer.send('news', {
                            'title': news_title,
                            'news_company': news_company,
                            'news_url': news_url,
                            'news_type': news_type[news_type_num],
                        })
                        producer.flush()
            news_type_num += 1

    print("time :", time.time() - start)


ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 99. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 99. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 99

KeyboardInterrupt: ignored